In [1]:
from ahrs.filters import Madgwick
from ahrs.common import Quaternion
import numpy as np
import h5py
import os
import matplotlib.pyplot as plt

In [2]:
def import_hdf5_sequences(file_path,):
    sample = {}
    with h5py.File(file_path, 'r') as f:
        
        data_dict = f["data"]
        for key in data_dict:
            sample[key] = data_dict[key][:]  
                    
        sample["participant"] = os.path.basename(file_path).split("_")[1]
        sample["session"] = os.path.basename(file_path).split("_")[2].replace(".hdf5", "")
        sample["label"] = [el.split("_")[0] for el in f.keys() if el != "data"]
        sample["label_times"] = [(f[key]["start_time"][()], f[key]["end_time"][()]) for key in f.keys() if key != "data"]
        sample["label_indices"] = [(f[key]["start_index"][()], f[key]["end_index"][()]) for key in f.keys() if key != "data"]
        sample["timestamp"] = [el.split("_")[1] for el in f.keys() if el != "data"]
    return sample

In [3]:
file_path = r"C:\Users\lhauptmann\Code\WristPPG2\data\dataset_corrected\participant_caroline\participant_caroline_1.hdf5"
samples = import_hdf5_sequences(file_path)

In [25]:
madgwick = Madgwick()

gyro_data = np.stack([samples["gyro_x"], samples["gyro_y"], samples["gyro_z"]], axis=1)
acc_data = np.stack([samples["acc_x"], samples["acc_y"], samples["acc_z"]], axis=1)
num_samples = gyro_data.shape[0]


Q = np.tile([1., 0., 0., 0.], (num_samples, 1)) # Allocate for quaternions
for t in range(1, num_samples):
    Q[t] = madgwick.updateIMU(Q[t-1], gyr=gyro_data[t], acc=acc_data[t])

In [37]:
Quaternion(Q[10000]).to_axang()

(array([-0.02827295, -0.87508169, -0.48314871]), 0.37594626756995103)